In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os 
import time

# Multiprocessing

## How `os.fork()` works?

<span style="font-family:New York Times; font-size:1em; color:green;">
    
os.fork() works by calling the underlying OS function fork()

With the return value of fork() we can decide in which process we are: 0 means that we are in the child process while a positive return value means that we are in the parent process. A negative return value means that an error occurred while trying to fork. 
    
Unix/Linux操作系统提供了一个fork()系统调用，它非常特殊。普通的函数调用，调用一次，返回一次，但是fork()调用一次，返回两次，因为操作系统自动把当前进程（称为父进程）复制了一份（称为子进程），然后，分别在父进程和子进程内返回。子进程永远返回0，而父进程返回子进程的ID。这样做的理由是，一个父进程可以fork出很多子进程，所以，父进程要记下每个子进程的ID，而子进程只需要调用getppid()就可以拿到父进程的ID。

In [4]:
print('Process (%s) start...' % os.getpid())

# Only works on Unix/Linux/Mac:
pid = os.fork() # Python的os模块封装了常见的系统调用，其中就包括fork，可以在Python程序中轻松创建子进程
print('pid = {}, os.getpid={}'.format(pid, os.getpid()))
if pid == 0:
    print('I am child process (%s) and my parent is %s.' % (os.getpid(), os.getppid()))
else:
    print('I (%s) just created a child process (%s).' % (os.getpid(), pid))
    #print('I (%s) just created a child process (%s).' % (os.getpid(), os.getpid()))

Process (61698) start...
pid = 63168, os.getpid=61698
I (61698) just created a child process (63168).
pid = 0, os.getpid=63168
I am child process (63168) and my parent is 61698.


[Liao XueFeng](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001431927781401bb47ccf187b24c3b955157bb12c5882d000)

In [17]:
from multiprocessing import Process

# 子进程要执行的代码
def run_proc(name): 
    print('Run child process {} {} ...'.format(name, os.getpid()))

if __name__=='__main__':
    print('Parent process %s.' % os.getpid())
    p = Process(target=run_proc, args=('TEST', ))
    print('Child process will start.')
    p.start()
    p.join()
    print('Child process end.')

Parent process 56804.
Child process will start.
Run child process TEST 56821 ...
Child process end.


In [18]:
def consumer():
    r = ''
    while True:
        n = yield r
        if not n:
            return
        print('[CONSUMER] Consuming %s...' % n)
        r = '200 OK'

def produce(c):
    c.send(None)
    n = 0
    while n < 5:
        n = n + 1
        print('[PRODUCER] Producing %s...' % n)
        r = c.send(n)
        print('[PRODUCER] Consumer return: %s' % r)
    c.close()

c = consumer()
produce(c)


[PRODUCER] Producing 1...
[CONSUMER] Consuming 1...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 2...
[CONSUMER] Consuming 2...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 3...
[CONSUMER] Consuming 3...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 4...
[CONSUMER] Consuming 4...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 5...
[CONSUMER] Consuming 5...
[PRODUCER] Consumer return: 200 OK


Multithreading in Python 

将进程挂起(Suspend) 而非 阻塞(Block)

如果用sleep() 进程将阻塞

假设进程下有两个线程 那么这两个线程会继续运行

Search for specific process in htop

In [20]:
from multiprocessing import Pool
import time, random

def long_time_task(name):
    print('Run task {} ({})...'.format(name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print('Task %s runs %0.2f seconds.' % (name, (end - start)))

if __name__=='__main__':
    print('Parent process %s.' % os.getpid())
    p = Pool(4)
    for i in range(5):
        p.apply_async(long_time_task, args=(i,))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')

Parent process 56804.


Run task 0 (56826)...


Run task 1 (56827)...


Run task 2 (56828)...


Run task 3 (56829)...
Waiting for all subprocesses done...
Task 2 runs 0.85 seconds.
Run task 4 (56828)...
Task 4 runs 1.10 seconds.
Task 0 runs 2.18 seconds.
Task 1 runs 2.41 seconds.
Task 3 runs 2.52 seconds.
All subprocesses done.


## [Python Multiprocessing: Pool vs Process – Comparative Analysis](https://www.ellicium.com/python-multiprocessing-pool-process/)

## 进程间通信
<span style="font-family:New York Times; font-size:1em; color:green;">
Process之间肯定是需要通信的，操作系统提供了很多机制来实现进程间的通信。Python的multiprocessing模块包装了底层的机制，提供了Queue、Pipes等多种方式来交换数据。

我们以Queue为例，在父进程中创建两个子进程，一个往Queue里写数据，一个从Queue里读数据：

In [ ]:
from multiprocessing import Process, Queue
import time, random

# 写数据进程执行的代码:
def write(q):
    print('Process to write: %s' % os.getpid())
    for value in ['A', 'B', 'C']:
        print('Put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())

# 读数据进程执行的代码:
def read(q):
    print('Process to read: %s' % os.getpid())
    while True:
        value = q.get(True)
        print('Get %s from queue.' % value)

if __name__=='__main__':
    # 父进程创建Queue，并传给各个子进程：
    q = Queue()
    pw = Process(target=write, args=(q,))
    pr = Process(target=read, args=(q,))
    # 启动子进程pw，写入:
    pw.start()
    # 启动子进程pr，读取:
    pr.start()
    # 等待pw结束:
    pw.join()
    # pr进程里是死循环，无法等待其结束，只能强行终止:
    pr.terminate()

# Multithreading

In [ ]:
from threading import Thread
from queue import Queue
class WorkerThread(Thread):
    def __init__(self,*args,**kwargs):
        Thread.__init__(self,*args,**kwargs)
        self.input_queue=Queue()

    def send(self,item):
        self.input_queue.put(item)
    def close(self):
        self.input_queue.put(None)
        self.input_queue.join()
    def run(self):
        while True:
            item=self.input_queue.get()
            if item is None:
                break
            #实际开发中，此处应该使用有用的工作代替
            print(item)
            self.input_queue.task_done()
        #完成，指示收到和返回哨兵
        self.input_queue.task_done()
        return

if __name__=="__main__":
    w=WorkerThread()
    w.start()
    w.send("Mark")
    w.send("好")
    w.send("？")
    w.close()



## `concurrent.futures`

* [PYTHON: A quick introduction to the concurrent to the `concurrent.futures`](http://masnun.com/2016/03/29/python-a-quick-introduction-to-the-concurrent-futures-module.html)

    The `concurrent.futures` module is part of the standard library which provides a high level API for launching async tasks
    
* [ThreadPoolExecutor线程池](https://www.jianshu.com/p/b9b3d66aa0be)
* [https://realpython.com/python-concurrency/](https://realpython.com/python-concurrency/)
* [http://python.jobbole.com/87272/](http://python.jobbole.com/87272/)

### `ThreadPoolExecutor`

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from time import sleep

def return_after_5_secs(message):
    sleep(5)
    return message

pool = ThreadPoolExecutor(3)
# pool.submit(func, (func_parameter)) 
future = pool.submit(return_after_5_secs, ("hello")) # submit tasks to the pool we constructed
print(future.done()) #  tells us if the future has resolved
sleep(5)
print(future.done())
print(future.result())

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import time

# 参数times用来模拟网络请求的时间
def get_html(times):
    time.sleep(times)
    print("get page {}s finished".format(times))
    return times

executor = ThreadPoolExecutor(max_workers=2)
# 通过submit函数提交执行的函数到线程池中，submit函数立即返回，不阻塞
task1 = executor.submit(get_html, (3))
task2 = executor.submit(get_html, (2))
# done方法用于判定某个任务是否完成
print(task1.done())
# cancel方法用于取消某个任务,该任务没有放入线程池中才能取消成功
print(task2.cancel())
time.sleep(4)
print(task1.done())
# result方法可以获取task的执行结果
print(task1.result())

# 执行结果
# False  # 表明task1未执行完成
# False  # 表明task2取消失败，因为已经放入了线程池中
# get page 2s finished
# get page 3s finished
# True  # 由于在get page 3s finished之后才打印，所以此时task1必然完成了
# 3     # 得到task1的任务返回值

### `as_completed`

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# 参数times用来模拟网络请求的时间
def get_html(times):
    time.sleep(times)
    print("get page {}s finished".format(times))
    return times

executor = ThreadPoolExecutor(max_workers=2)
urls = [3, 2, 4] # 并不是真的url
all_task = [executor.submit(get_html, (url)) for url in urls]

for future in as_completed(all_task):
    data = future.result()
    print("in main: get page {}s success".format(data))

# 执行结果
# get page 2s finished
# in main: get page 2s success
# get page 3s finished
# in main: get page 3s success
# get page 4s finished
# in main: get page 4s success

### `wait`

<span style="font-family:New York Times; font-size:1em; color:green;">wait方法接收3个参数，等待的任务序列、超时时间以及等待条件。等待条件return_when默认为ALL_COMPLETED，表明要等待所有的任务都结束。可以看到运行结果中，确实是所有任务都完成了，主线程才打印出main。等待条件还可以设置为FIRST_COMPLETED，表示第一个任务完成就停止等待。

In [ ]:
from concurrent.futures import ThreadPoolExecutor, wait, ALL_COMPLETED, FIRST_COMPLETED
import time

# 参数times用来模拟网络请求的时间
def get_html(times):
    time.sleep(times)
    print("get page {}s finished".format(times))
    return times

executor = ThreadPoolExecutor(max_workers=2)
urls = [3, 2, 4] # 并不是真的url
all_task = [executor.submit(get_html, (url)) for url in urls]
wait(all_task, return_when=ALL_COMPLETED)
print("main")
# 执行结果 
# get page 2s finished
# get page 3s finished
# get page 4s finished
# main

In [ ]:
def get_html(times):
    time.sleep(times)
    print("get page {}s finished".format(times))
    return times

executor = ThreadPoolExecutor(max_workers=2)
urls = [3, 2, 4] # 并不是真的url
all_task = [executor.submit(get_html, (url)) for url in urls]
wait(all_task, return_when=FIRST_COMPLETED)
print("main")

https://stackoverflow.com/questions/57600892/why-is-python-multithreading-in-this-example-so-slow

In [2]:
import time

start_time = time.time()


def testThread(num):
    num = ""
    for i in range(500):
        num += str(i % 10)
        a.write(num)


def main():
    for i in range(3000):
        testThread(i)


if __name__ == '__main__':
    a = open('single.txt', 'w')
    main()
    print(time.time() - start_time)

1.3276479244232178


In [3]:
import threading
from concurrent.futures import ThreadPoolExecutor
from multiprocessing.pool import ThreadPool
import time

start_time = time.time()


def testThread(num):
    num = ""
    for i in range(500):
        num += str(i % 10)
        with global_lock:
            a.write(num)


def main():
    test_list = [x for x in range(3000)]

    with ThreadPool(4) as executor:
        results = executor.map(testThread, test_list)

    # with ThreadPoolExecutor() as executor:
    #    results = executor.map(testThread, test_list)


if __name__ == '__main__':
    a = open('multi.txt', 'w')
    global_lock = threading.Lock()
    main()
    print(time.time() - start_time) 

6.866662263870239


https://stackoverflow.com/questions/57618697/time-sleep-not-working-as-expected-when-adding-end-to-print

In [14]:
import time,threading

msg = "68 111 110 117 116 115 32 97 114 101 32 98 111 109 98 "
msg = [int(x) for x in msg.split()]

def print_msg():
    for c in msg:
        print(chr(c)),time.sleep(0.2)

threading.Thread(target=print_msg).start()

D


In [15]:
import time, threading

msg = "68 111 110 117 116 115 32 97 114 101 32 98 111 109 98 "
msg = [int(x) for x in msg.split()]

def print_msg():
    for c in msg:
        print(chr(c), end="", flush=True)
        time.sleep(0.2)

threading.Thread(target=print_msg).start()

u
D

# When do we need parallel computation

<span style="font-family:New York Times; font-size:1em; color:green;">
    
we would want to use the `ProcessPoolExecutor` for CPU intensive tasks. The `ThreadPoolExecutor` is better suited for network operations or I/O.

In [ ]:
import queue
import threading
import urllib.request, urllib.error, urllib.parse

# called by each thread
def get_url(q, url):
    q.put(urllib.request.urlopen(url).read())

theurls = ["http://google.com", "http://yahoo.com"]

q = queue.Queue()

for u in theurls:
    t = threading.Thread(target=get_url, args = (q,u))
    t.daemon = True
    t.start()

s = q.get()
print(s)

In [ ]:
Result = [('80407', 'about power supply of opertional amplifier', '11 hours ago'), ('80405', '5V Regulator Power Dissipation', '11 hours ago')]

In [ ]:
Result[0]

In [ ]:
print('\n'.join(';'.join(i) for i in Result))

In [ ]:
def findClosestPair(arr0):
    arr0s = sorted(arr0)
    n = len(arr0)
    z = []
    x = 0 

    if n != len(set(arr0s)):
        return ("No repeated elements")
    else: 
        while x<n-2:
            if arr0s[x+1]-arr0s[x] < 20:
                if arr0s[x+1]-arr0s[x] < arr0s[x+2]-arr0s[x+1]:
                    z.append([arr0s[x], arr0s[x+1]])
                    x+=2 
                else:
                    z.append([arr0s[x+1], arr0s[x+2]])
                    x+=3
            else:
                x+=1 
        # from value in z, find the corresponding index in arr0
        result_indexes = [[arr0.index(i[0]), arr0.index(i[1])]  for i in z] 
        # Adjust the index order
        for i, j in enumerate(result_indexes):
            if j[0]>j[1]:
                result_indexes[i] = [j[1], j[0]]

        result_value = [[arr0[i[0]], arr0[i[1]]] for i in result_indexes]
        return (result_indexes,result_value )

arr0 = [40, 55, 190, 80, 175, 187]
findClosestPair(arr0)

In [ ]:

arr0s = sorted(arr0)
n = len(arr0)
z = []
x = 0 
while x<n-2:
    if arr0s[x+1]-arr0s[x] < 20:
        if arr0s[x+1]-arr0s[x] < arr0s[x+2]-arr0s[x+1]:
            z.append([arr0s[x], arr0s[x+1]])
            x+=2 
        else:
            z.append([arr0s[x+1], arr0s[x+2]])
            x+=3
    else:
        x+=1 
    result_indexes = [[arr0.index(i[0]), arr0.index(i[1])]  for i in z] 

    for i, j in enumerate(result_indexes):
        if j[0]>j[1]:
            result_indexes[i] = [j[1], j[0]]
    result_value = [[arr0[i[0]], arr0[i[1]]] for i in result_indexes]
print(result_indexes)
#[[0, 1], [2, 5]]
print(result_value)
#[[40, 55], [190, 187]]

## Timed out input

In [ ]:
import time
from threading import Thread

answer = None

def check():
    time.sleep(2)
    if answer != None:
        return
    print ("Too Slow")

Thread(target = check).start()

answer = input("Input something: ")

In [ ]:
a = 3
pw = 'pw'


t_end = time.time() + 5  
print("Inter the pass word")
while time.time() < t_end:
    what = input(":")
    if pw == what:
        print(a)
        break 
        
    else:
        print("Wrong, Input again!")
        continue
    
print("Time is up")



In [ ]:
try:
    while True:
        data = eval(input('prompt:'))
        print('READ:', data)
except EOFError as e:
    print(e)